In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick
import pgeocode
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor

In [ ]:
# Read HouseTS.csv into area_df
area_df = pd.read_csv('../raw_data/HouseTS.csv')

# Read realtor-data.csv into house_df
house_df = pd.read_csv('../raw_data/realtor-data.csv')

In [ ]:
# Create list of unique zipcodes in area_df
unique_zipcodes_area_df = area_df['zipcode'].unique().tolist()

# Filter house_df by unique_zipcoes_area_df
house_df = house_df[house_df['zip_code'].isin(unique_zipcodes_area_df)]

CPU times: user 95 μs, sys: 21 μs, total: 116 μs
Wall time: 121 μs


,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
0,103378.0,for_sale,105000.0,3.0,2.0,0.12,1962661.0,Adjuntas,Puerto Rico,601.0,920.0,NaN
1,52707.0,for_sale,80000.0,4.0,2.0,0.08,1902874.0,Adjuntas,Puerto Rico,601.0,1527.0,NaN
2,103379.0,for_sale,67000.0,2.0,1.0,0.15,1404990.0,Juana Diaz,Puerto Rico,795.0,748.0,NaN
3,31239.0,for_sale,145000.0,4.0,2.0,0.10,1947675.0,Ponce,Puerto Rico,731.0,1800.0,NaN
4,34632.0,for_sale,65000.0,6.0,2.0,0.05,331151.0,Mayaguez,Puerto Rico,680.0,NaN,NaN


In [ ]:
def clean_data(df):
    # Drop columns 'brokered_by', 'status'
    df = df.drop(columns=['brokered_by', 'status'])

     # Drop duplicates
    df = df.drop_duplicates()

    # Drop columns 'street', 'city', 'state' and 'prev_sold_date'
    df = df.drop(columns=['street', 'city', 'state', 'prev_sold_date'])

    # Drop rows with NaN values from 'price'
    df = df.dropna(subset=['price'])

    # Create list where 'bed' & 'bath' & 'house_size' are NaN
    nan_values = df[
        (pd.isna(df['bed'])) &
        (pd.isna(df['bath'])) &
        (pd.isna(df['house_size']))
    ]

    # Filter out rows that are in nan_values because we assume they are land sales
    df = df[~df.index.isin(nan_values.index)]

    # Impute missing data
    df['bed'] = df['bed'].fillna(df['bed'].median())
    df['bath'] = df['bath'].fillna(df['bath'].median())
    df['house_size'] = df['house_size'].fillna(df['house_size'].median())
    df['acre_lot'] = df['acre_lot'].fillna(0)

    # Step 2: Calculate PPSF for each row
    df['ppsf'] = df['price'] / df['house_size']

    # Step 3: Calculate median PPSF per zip_code
    ppsf_median = df.groupby('zip_code')['ppsf'].median().reset_index(name='ppsf_zipcode')

    # Step 4: Merge median PPSF back to df
    df = df.merge(ppsf_median, on='zip_code', how='left')

    # Drop temporary ppsf column
    df = df.drop(columns=['ppsf'])

    # Calculate boundaries for 'price', 'acre_lot', 'house_size', 'ppsf_zipcode'
    lower_price = df['price'].quantile(0.03)
    upper_price = df['price'].quantile(0.97)
    upper_house_size = df['house_size'].quantile(0.99)
    lower_acre_lot = df['acre_lot'].quantile(0.01)
    upper_acre_lot = df['acre_lot'].quantile(0.99)
    lower_ppsf_zipcode = df['ppsf_zipcode'].quantile(0.03)
    upper_ppsf_zipcode = df['ppsf_zipcode'].quantile(0.97)

    # Apply boundaries to df
    df = df[
        (df['price'] > lower_price) &
        (df['price'] < upper_price) &
        (df['bed'] < 14) &
        (df['bath'] < 12) &
        (df['house_size'] < upper_house_size) &
        (df['acre_lot'] > lower_acre_lot) &
        (df['acre_lot'] < upper_acre_lot) &
        (df['ppsf_zipcode'] > lower_ppsf_zipcode) &
        (df['ppsf_zipcode'] < upper_ppsf_zipcode)
        ]

    return df

In [ ]:
# Clean df
cleaned_house_df = clean_data(house_df)

In [ ]:
def convert_zipcode(df):
    # Convert zip_code column to 5-digit string
    df['zip_code'] = df['zip_code'].astype(str).str.replace('\.0$', '', regex=True).str.zfill(5)

    # Get unique zip codes
    unique_zips = df['zip_code'].unique()

    # Initialize pgeocode for US
    nomi = pgeocode.Nominatim('us')

    # Function to get coordinates
    def get_coordinates(zip_code):
        try:
            result = nomi.query_postal_code(zip_code)
            if result.empty or pd.isna(result.latitude):
                return pd.Series([None, None])
            return pd.Series([result.latitude, result.longitude])
        except:
            return pd.Series([None, None])

    # Create DataFrame for unique zip codes
    zip_coords = pd.DataFrame(unique_zips, columns=['zip_code'])
    zip_coords[['latitude', 'longitude']] = zip_coords.apply(lambda row: get_coordinates(row['zip_code']), axis=1)

    # Map coordinates back to filtered_house_df
    coords_dict = zip_coords.set_index('zip_code')[['latitude', 'longitude']].to_dict('index')
    df['latitude'] = df['zip_code'].map(lambda x: coords_dict.get(x, {}).get('latitude'))
    df['longitude'] = df['zip_code'].map(lambda x: coords_dict.get(x, {}).get('longitude'))

    # Drop 'zip_code' column
    df = df.drop(columns=['zip_code'])

    return df

<class 'pandas.core.frame.DataFrame'>
Index: 936724 entries, 5199 to 2224341
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   brokered_by     935597 non-null  float64
 1   status          936724 non-null  object 
 2   price           936401 non-null  float64
 3   bed             844470 non-null  float64
 4   bath            833568 non-null  float64
 5   acre_lot        755663 non-null  float64
 6   street          931960 non-null  float64
 7   city            936587 non-null  object 
 8   state           936724 non-null  object 
 9   zip_code        936724 non-null  float64
 10  house_size      793101 non-null  float64
 11  prev_sold_date  735667 non-null  object 
dtypes: float64(8), object(4)
memory usage: 92.9+ MB


In [ ]:
# Convert zipcodes to coordinates
cleaned_house_df = convert_zipcode(cleaned_house_df)
#cleaned_house_df

In [ ]:
cleaned_house_df.describe()

             price  bed  bath  acre_lot     street        city          state  \
24595     659900.0  NaN   NaN    106.86  1853232.0  Barrington  New Hampshire   
44736     449000.0  5.0   3.0      0.06   893062.0  Bloomfield     New Jersey   
46234     329999.0  4.0   2.0      0.03  1260708.0     Haledon     New Jersey   
46583     524900.0  5.0   3.0      0.06   984852.0        Lodi     New Jersey   
46767     499000.0  5.0   3.0      0.05  1372844.0    Brooklyn       New York   
...            ...  ...   ...       ...        ...         ...            ...   
2197155  1025000.0  3.0   2.0      0.03  1374909.0     Seattle     Washington   
2197253   680000.0  3.0   2.0      0.08   111278.0      Renton     Washington   
2197789   775000.0  3.0   2.0      0.05   194389.0     Seattle     Washington   
2202651   549500.0  5.0   3.0      0.08   133822.0    Puyallup     Washington   
2203665  1897000.0  3.0   3.0      0.48  1146365.0   Sammamish     Washington   

         zip_code  house_si

In [ ]:
# # Work on a copy to avoid SettingWithCopyWarning
# df = cleaned_house_df.copy()
# df = df.drop(columns=['latitude', 'longitude'])

# # Define features and target
# target = 'price'
# features = [col for col in df.columns if col != target]  # Exclude price
# numeric_features = [col for col in features if col != 'zip_code']  # Exclude zip_code

# # Verify columns
# print("\nFeatures:", features)
# print("Numeric features for scaling:", numeric_features)
# if target not in df.columns:
#     raise ValueError(f"'{target}' column not found. Available columns: {df.columns.tolist()}")

# # Create X and y
# X = df[features]
# y = df[target]

# # Preprocess with ColumnTransformer
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', StandardScaler(), numeric_features)
#     ],
#     remainder='passthrough'  # Keep zip_code unscaled
# )

# # Split data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Create pipeline
# from sklearn.pipeline import Pipeline
# pipeline = Pipeline([
#     ('preprocessor', preprocessor),
#     ('regressor', XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42, n_jobs=-1))
# ])

# # Train model
# pipeline.fit(X_train, y_train)

# # Predict and evaluate
# y_pred = pipeline.predict(X_test)

# # Calculate metrics
# rmse = np.sqrt(mean_squared_error(y_test, y_pred))
# mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# # Print results
# print("\nModel Results:")
# print(f"XGBoost RMSE: ${rmse:,.2f}")
# print(f"Mean Absolute Error (MAE): ${mae:,.2f}")
# print(f"R² Score: {r2:.4f}")

# # Feature importance
# feature_names = numeric_features + ['zip_code']
# print("\nFeature Importance:")
# for name, importance in zip(feature_names, pipeline.named_steps['regressor'].feature_importances_):
#     print(f"{name}: {importance:.4f}")

# # Sample of actual vs. predicted prices
# results_df = pd.DataFrame({
#     'Actual Price': y_test,
#     'Predicted Price': y_pred,
#     'Difference': y_test - y_pred
# })
# print("\nSample of Actual vs. Predicted Prices:")
# print(results_df.head(10))


In [10]:
# import pandas as pd
# import numpy as np
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# from sklearn.pipeline import Pipeline
# from xgboost import XGBRegressor

# # 1. Work on a copy to avoid SettingWithCopyWarning
# df = cleaned_house_df.copy()
# df = df.drop(columns=['latitude', 'longitude'])

# # 2. Define features and target
# target = 'price'
# features = [col for col in df.columns if col != target]  # Exclude price
# numeric_features = [col for col in features if col != 'zip_code']  # Exclude zip_code

# # Verify columns
# print("\nFeatures:", features)
# print("Numeric features for scaling:", numeric_features)
# if target not in df.columns:
#     raise ValueError(f"'{target}' column not found. Available columns: {df.columns.tolist()}")

# # 3. Create X and y
# X = df[features]
# y = df[target]

# # 4. Preprocess with ColumnTransformer
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', StandardScaler(), numeric_features)  # Scale numeric features
#     ],
#     remainder='passthrough'  # Keep zip_code unscaled
# )

# # 5. Split data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # 6. Create pipeline with XGBoost
# pipeline = Pipeline([
#     ('preprocessor', preprocessor),
#     ('regressor', XGBRegressor(random_state=42, n_jobs=-1))
# ])

# # 7. Define hyperparameter grid for GridSearchCV
# param_grid = {
#     'regressor__n_estimators': [100, 200],  # Number of trees
#     'regressor__learning_rate': [0.01, 0.1],  # Step size for boosting
#     'regressor__max_depth': [3, 5],  # Depth of trees
#     'regressor__min_child_weight': [1, 3]  # Minimum sum of instance weight needed in a child
# }

# # 8. Perform GridSearchCV
# grid_search = GridSearchCV(
#     pipeline,
#     param_grid,
#     cv=3,  # 3-fold cross-validation
#     scoring='neg_mean_squared_error',
#     n_jobs=-1,  # Use all cores
#     verbose=1
# )

# # 9. Fit the model
# grid_search.fit(X_train, y_train)

# # 10. Evaluate
# best_model = grid_search.best_estimator_
# y_pred = best_model.predict(X_test)

# rmse = np.sqrt(mean_squared_error(y_test, y_pred))
# mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# # Print results
# print("\nModel Results:")
# print(f"Best parameters: {grid_search.best_params_}")
# print(f"Best cross-validated RMSE: ${np.sqrt(-grid_search.best_score_):,.2f}")
# print(f"Test RMSE: ${rmse:,.2f}")
# print(f"Mean Absolute Error (MAE): ${mae:,.2f}")
# print(f"R² Score: {r2:.4f}")

# # 11. Feature importance
# feature_names = numeric_features + ['zip_code']
# regressor = best_model.named_steps['regressor']
# print("\nFeature Importance:")
# for name, importance in zip(feature_names, regressor.feature_importances_):
#     print(f"{name}: {importance:.4f}")

# # 12. Sample of actual vs. predicted prices
# results_df = pd.DataFrame({
#     'Actual Price': y_test,
#     'Predicted Price': y_pred,
#     'Difference': y_test - y_pred
# })
# print("\nSample of Actual vs. Predicted Prices:")
# print(results_df.head(10))

In [16]:
print("test")

test


In [15]:
# Work on a copy to avoid SettingWithCopyWarning
df = cleaned_house_df.copy()
df = df.drop(columns=['latitude', 'longitude'])

# Define features and target
target = 'price'
features = [col for col in df.columns if col != target]  # Exclude price
numeric_features = [col for col in features if col != 'zip_code']  # Exclude zip_code

# Verify columns
print("\nFeatures:", features)
print("Numeric features for scaling:", numeric_features)
if target not in df.columns:
    raise ValueError(f"'{target}' column not found. Available columns: {df.columns.tolist()}")

# Create X and y
X = df[features]
y = df[target]

# Preprocess with ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features)
    ],
    remainder='passthrough'  # Keep zip_code unscaled
)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create pipeline
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42, n_jobs=-1))
])

# Train model
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)

# Calculate metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print results
print("\nModel Results:")
print(f"XGBoost RMSE: ${rmse:,.2f}")
print(f"Mean Absolute Error (MAE): ${mae:,.2f}")
print(f"R² Score: {r2:.4f}")

# Feature importance
feature_names = numeric_features + ['zip_code']
print("\nFeature Importance:")
for name, importance in zip(feature_names, pipeline.named_steps['regressor'].feature_importances_):
    print(f"{name}: {importance:.4f}")

# Sample of actual vs. predicted prices
results_df = pd.DataFrame({
    'Actual Price': y_test,
    'Predicted Price': y_pred,
    'Difference': y_test - y_pred
})
print("\nSample of Actual vs. Predicted Prices:")
print(results_df.head(10))


935332